
# Stonks
## Note: Interactive Plotly figures wont show up when viewing at Github, but trust me, they're neat
## Import needed packages

In [65]:
import pandas as pd
import numpy as np
import datetime as dt
import pandas_datareader.data as pdr
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from tqdm.notebook import tqdm
# yf.pdr_override()


## Functions to get stock data and to plot share prices

In [66]:
def get_fundamentals(tickers):
    '''Gets the fundamentals data for given tickers and produces a clean dataframe from it'''
    
    tickers_data = {}
    fundamentals = ['forwardPE',
                    'trailingPE',
                    'forwardEps',
                    'sector',
                    'fullTimeEmployees',
                    'country',
                    'twoHundredDayAverage',
                    'averageDailyVolume10Day',
                    'trailingPE',
                    'marketCap',
                    'priceToSalesTrailing12Months',
                    'trailingEps',
                    'priceToBook',
                    'earningsQuarterlyGrowth',
                    'pegRatio']
    # Loop all tickers and get some interesting fundamentals.
    for ticker in tqdm(tickers):
        ticker_object = yf.Ticker(ticker)

        #convert info() output from dictionary to dataframe
        new_info = { key:value for (key,value) in ticker_object.info.items() if key in fundamentals}
        temp = pd.DataFrame.from_dict(new_info, orient="index")
        temp.reset_index(inplace=True)
        if len(temp.columns) == 2:
            temp.columns = ["Attribute", "Value"]
            # add (ticker, dataframe) to main dictionary
            tickers_data[ticker] = temp


    combined_data = pd.concat(tickers_data).reset_index().drop(columns="level_1").rename(columns={'level_0': 'Ticker'})
    combined_data = combined_data.pivot(index='Ticker', columns='Attribute', values='Value').reset_index()
    combined_data = combined_data.rename_axis(None, axis=1).infer_objects()
    combined_data.dropna(inplace=True) # Drop if any fundamental is NA
    return combined_data

In [67]:
def get_data(mode="test"):
    '''Fetches stock tickers and fundamentals data from Yahoo or csv'''
    if mode == "test":
        # Tickers for lighter computing
        tickers =['FB','AMZN', 'AAPL', 'NFLX', 'GOOGL', 'MSFT']
        fundamentals = get_fundamentals(tickers)
    elif mode == "all":
        #Get all tickers from csv, if no csv in directory -> scrape them from wikipedia
        SP500_fileName = "SP500_symbols.csv"
        if not os.path.isfile(SP500_fileName):
            tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
            tickers = tickers[0]["Symbol"]
            tickers.to_csv(SP500_fileName)
        else:
            tickers = pd.read_csv(SP500_fileName).drop(['Unnamed: 0'],axis=1)

        # Get all fundamentals from csv, if no csv in directory -> scrape them from yahoo
        fundamentals_fileName = "SP500_fundamentals.csv"
        if not os.path.isfile(fundamentals_fileName):
            fundamentals = get_fundamentals(tickers)
            fundamentals.to_csv(fundamentals_fileName)
        else:
            fundamentals = pd.read_csv(fundamentals_fileName).drop(['Unnamed: 0'],axis=1)
    else:
        print("Select mode")
        return 0

    return tickers,fundamentals[fundamentals["Ticker"] != "UDR"] # Remove UDR from data as a huge outlier


In [68]:
def monitor_stock(stockName,start_date = "2020-01-01"):
    '''Creates an interactive Plotly figure to monitor the share prices and volumes of given stocks'''

    start = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end = dt.datetime.now()
    stock_df = pdr.DataReader(stockName, 'yahoo', start, end)
    # stocks.describe()
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, 
               row_width=[0.2, 0.7])

    # Old, used when there are multiple stocks in the df
    # fig.add_trace(go.Candlestick(x = stock_df.index, 
    #                                                open = stock_df[('Open',    stockName)], 
    #                                                high = stock_df[('High',    stockName)], 
    #                                                low = stock_df[('Low',    stockName)], 
    #                                                close = stock_df[('Close',    stockName)],showlegend=False,name="Price"))


    fig.add_trace(go.Candlestick(x = stock_df.index, 
                                                   open = stock_df['Open'], 
                                                   high = stock_df['High'], 
                                                   low = stock_df['Low'], 
                                                   close = stock_df['Close'],showlegend=False,name="Price"))

    fig.update_xaxes(row=1, col=1,
        title_text = '',
        rangeslider_visible = False,
        rangeselector = dict(
            buttons = list([
                dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
                dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
                dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
                dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
                dict(step = 'all')])))
    
    fig.add_trace(go.Bar(x = stock_df.index,
                        y=stock_df['Volume'],
                        showlegend=False,name="Volume",
                        marker=dict(color="rgba(0,0,0.8,0.66)")),row=2, col=1)

 
    
    fig.update_layout(
        width=1280,
        height=800,
        title = {
            'text': stockName +' STOCK MONITOR',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            plot_bgcolor =  "rgba(1,1,1,0.05)")
    
    fig.update_yaxes(title_text ='Close Price', tickprefix = '$',row=1,col=1)
    fig.update_yaxes(title_text = 'Volume',row=2,col=1)
    fig.show()

## Monitor one example stock

In [69]:
monitor_stock("GOOGL")

## Get fundamentals data for companies

In [70]:
# tickers,fundamentals = get_data("test")
tickers,fundamentals = get_data("all")
# fundamentals = pd.read_csv("SP500_fundamentals.csv").drop(['Unnamed: 0'],axis=1)
# fundamentals.columns
fundamentals


,Ticker,averageDailyVolume10Day,country,earningsQuarterlyGrowth,forwardEps,forwardPE,fullTimeEmployees,marketCap,pegRatio,priceToBook,priceToSalesTrailing12Months,sector,trailingEps,trailingPE,twoHundredDayAverage
0,A,1396414.0,United States,0.462,4.36,30.917430,16400.0,4.107329e+10,2.91,8.555471,7.427358,Healthcare,2.597,51.906048,119.774414
1,AAP,920185.0,United States,0.168,11.66,16.949400,40000.0,1.289753e+10,1.57,3.654468,1.276185,Consumer Cyclical,7.140,27.679274,164.982210
2,AAPL,79115057.0,United States,0.293,4.72,28.300850,147000.0,2.242554e+12,2.00,33.938007,7.624235,Technology,3.687,36.229996,125.668380
3,ABBV,4930885.0,United States,-0.987,13.87,8.069935,47000.0,1.975374e+11,2.15,15.109342,4.312667,Healthcare,2.720,41.150734,103.048010
4,ABMD,231200.0,United States,-0.106,4.91,71.521390,1536.0,1.588377e+10,4.02,12.589898,19.538795,Healthcare,4.387,80.047870,304.002440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,XLNX,2203175.0,United States,0.055,3.30,39.854546,4891.0,3.232223e+10,5.20,12.408718,10.587797,Technology,2.499,52.629050,129.577700
357,XYL,849575.0,United States,0.254,3.09,36.006474,15600.0,2.002702e+10,2.34,6.760649,4.107264,Industrials,1.400,79.471430,96.721260
358,ZBRA,271150.0,United States,0.178,16.95,29.297934,8800.0,2.656775e+10,3.02,12.383114,5.972966,Technology,9.350,53.112297,381.658600
359,ZION,1163275.0,United States,22.000,4.33,12.759815,9682.0,9.049950e+09,-0.32,1.228433,3.232125,Financial Services,4.919,11.231957,43.150295


# Fundamental analysis on the companies
We'll use the following key fundamental indicators:   
   
__Price-to-earnings ratio (P/E):__ This ratio compares the current sales price of a company's stock to its per-share earnings. This should be used in comparison to other stocks, and not as an absolute value.     
   
__Projected earnings growth (PEG):__ PEG anticipates the one-year earnings growth rate of the stock. General rule: <1 => undervalued or ok. PEG = P/E ratio is divided by expected earnings growth going forward. A ratio significantly above 1.0 indicates that the stock might be too expensive for its likely growth. You’ll spend a lot of money but probably won’t see much return. A ratio much below 1.0 can indicate that the stock is poised for real growth, with its price too low for the company’s likely performance.      
   
__Price-to-sales ratio (P/S):__ The price-to-sales ratio values a company's stock price as compared to its revenues. The Price to Sales ratio is simply: Price divided by Sales. If the Price to Sales ratio is 1, then you're paying \$1 for every \$1 of sales the company makes. The lower the Price the Sales ratio, the better.    
   
__Price-to-book ratio (P/B):__ This ratio, also known as the price-to-equity ratio, compares a stock's book value to its market value. You can arrive at it by dividing the stock's most recent closing price by last quarter's book value per share. Book value is the value of an asset, as it appears in the company's books. It's equal to the cost of each asset less cumulative depreciation. Looking at the P/B ratio of a stock can be beneficial from a value investing perspective if you’re trying to find the undervalued hidden gems of the market. When companies have a low price-to-book ratio, meaning they’re trading for less than their book value, it can mean that the market has underestimated what the company is worth.   
   
__Forward earnings per share (EPS):__ While earnings are a company’s revenue minus operation expenses, earnings per share are the earnings remaining for shareholders divided by the number of outstanding shares. If a company has high earnings per share, investors perceive them to be more profitable.   
   

In [71]:
funs_to_use = ['Ticker','forwardPE','pegRatio','forwardEps','priceToSalesTrailing12Months','priceToBook']
funs = fundamentals[funs_to_use]
funs

# funs = fundamentals.loc[fundamentals["sector"]=="Technology",funs_to_use]
# funs

,Ticker,forwardPE,pegRatio,forwardEps,priceToSalesTrailing12Months,priceToBook
0,A,30.917430,2.91,4.36,7.427358,8.555471
1,AAP,16.949400,1.57,11.66,1.276185,3.654468
2,AAPL,28.300850,2.00,4.72,7.624235,33.938007
3,ABBV,8.069935,2.15,13.87,4.312667,15.109342
4,ABMD,71.521390,4.02,4.91,19.538795,12.589898
...,...,...,...,...,...,...
356,XLNX,39.854546,5.20,3.30,10.587797,12.408718
357,XYL,36.006474,2.34,3.09,4.107264,6.760649
358,ZBRA,29.297934,3.02,16.95,5.972966,12.383114
359,ZION,12.759815,-0.32,4.33,3.232125,1.228433


## Box plots of the chosen fundamentals

In [72]:
fig = px.box(funs.melt(id_vars=["Ticker"]),
 y="value",
  facet_col="variable",
   color="variable",
   boxmode="overlay",
   hover_name="Ticker")

fig.update_layout(width=1280,
                    height=600,
                    showlegend=False)
fig.update_yaxes(matches=None,showticklabels=True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()


The current forward P/E of the whole SP500 list of companies is around USD 29 (May 2021), which is probably slightly inflated due to the current pandemic. We'll see what companies have their P/E ratio under USD 20, as they could be considered "cheaper" than the market. The PEG ratio being significantly under 1 is a sign of its price being too low for the company’s likely performance in the future.
   
Using these information, we can filter some promising companies from the SP500 list. We can then afterward visualise these companies using their respective values for EPS, P/S and P/B.


In [73]:
PE_filter = 20
peg_filter = 1
funs_filtered = funs.query('forwardPE <= @PE_filter & pegRatio <= @peg_filter')

fig = px.scatter(funs_filtered, x="forwardEps", y="priceToBook",
                 size='priceToSalesTrailing12Months', hover_name='Ticker',hover_data=['forwardPE','pegRatio'])
fig.update_layout(
        width=1280,
        height=800,
        title = {
            'text': f'SP500 companies with forwardPE <= {PE_filter} & pegRatio <= {peg_filter}',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})

fig.add_annotation(
  x=30,  # arrows' head
  y=5,  # arrows' head
  ax=15,  # arrows' tail
  ay=20,  # arrows' tail
  xref='x',
  yref='y',
  axref='x',
  ayref='y',
  text='High EPS and low P/B could indicate promising companies',
  showarrow=True,
  arrowhead=3,
  arrowsize=4,
  arrowwidth=1,
  arrowcolor='black'
)
fig.show()

In [74]:
# More analysis here; try to get WACC, DCF or FCF
# set(test.recommendations["To Grade"].values)

## Perform PCA

In [75]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import random

## Correlation matrix of fundamentals

In [76]:
funs.corr()

,forwardPE,pegRatio,forwardEps,priceToSalesTrailing12Months,priceToBook
forwardPE,1.000000,-0.048078,-0.071815,0.715427,0.081142
pegRatio,-0.048078,1.000000,-0.001921,0.087960,-0.001040
forwardEps,-0.071815,-0.001921,1.000000,-0.036922,0.038919
priceToSalesTrailing12Months,0.715427,0.087960,-0.036922,1.000000,0.130314
priceToBook,0.081142,-0.001040,0.038919,0.130314,1.000000


# PCA

In [77]:
def pca_on_fundamentals(data):
    '''Performs PCA on the numeric values of the fundamentals dataset'''
    features = data.select_dtypes(include=np.number).columns.tolist()
    x = data.loc[:, features].values
    x = StandardScaler().fit_transform(x)
    pd.DataFrame(data = x, columns = features).head()
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2'])
    # print("Explained variance ratios: ",pca.explained_variance_ratio_)
    return principalDf

def plot_pca(data):
    '''Plots the PCA onto two dimensions using interactive Plotly scatterplot'''
    principalDf = pca_on_fundamentals(data)
    # rand_colours = color = [
    #     "#" + "".join([random.choice("0123456789ABCDEF") for j in range(6)])
    #     for i in range(len(pd.unique(data['sector'])))
    # ]
    # colours = {i:rand_colours[k] for k,i in enumerate(pd.unique(data['sector']))}

    # col_df = pd.DataFrame.from_dict(colours,orient='index').reset_index()
    # col_df.columns = ["sector", "Colour"]
    # t = pd.merge(data,col_df,how="left")

    value_for_colour = 'forwardPE'
    fig = go.Figure(go.Scatter(
        x=principalDf["PC1"],
        y=principalDf["PC2"],
        mode='markers',
        text=data["Ticker"],
        marker_colorbar=dict(thickness=10,title=value_for_colour),
        marker_color=data[value_for_colour]
        ))

    fig.update_layout(
        width=1280,
        height=800,
        xaxis_title="PC1 Score",
        yaxis_title="PC2 Score",
        title = {
            'text': 'PCA of SP500 companies',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            plot_bgcolor =  "rgba(1,1,1,0.05)")

    fig.show()

In [78]:
# pca = pca_on_fundamentals(fundamentals)
plot_pca(funs)

PCA's first two dimensions explain roughly 55% of the variation in the data. We should look if we could use Tensorflow to analyse the data, as the data could have highly nonlinear and complex structure.

# Tensorflow
## Create an classification tool with Tensorflow and try to classify companies to the categories given by "professional" analysts
### Also, switch to Seaborn for plotting

In [79]:
import seaborn as sns
import matplotlib.pyplot as plt

### Get analysts' recommendations and give each stock a class label using "majority-rule" on the recommendations.

### Preprocess data

### Train the model

### Model validation

### Test the model